In [1]:
from datasets import Dataset, DatasetDict, Image
import os
import json

data_dict = {
    'scene_name': [],
    'ground_truth': [],
    'caption': [],
    'conditioning_images_one': [],
    'conditioning_images_two': [],
    'reference_image': []
}

# This function can be used for edges rgb seg
def get_image_paths_prescan(dir, indexes, type):
    image_paths = []
    out_of_range = False
    for index in indexes:
        # Before we return we need to sort all the images, but the naming convention is different per type
        # files names
        if type == "edges" or type == "rgb":
            # The filename is CameraSensor_1_00000
            file_name = "CameraSensor_1_"
            index = str(index).zfill(5)
        if type == "seg":
            # The filename is ImageSegmentationSensor_1_00000
            file_name = "ImageSegmentationSensor_1_"
            index = str(index).zfill(5)
        if type == "caption":
            # The filename is caption.txt
            file_name = "caption"
            index = ""
        if type == "ground_truth":
            # The filename is frame0
            file_name = "frame"
            index = str(index)
        try:
            image_paths.append(os.path.join(dir, f"{file_name}{index}.png"))
        except:
            out_of_range = True
            break
    
    # Now we need to sort the imagesPahts

    if type == "edges" or type == "rgb" or type == "seg":
        # the names are cameraSensor_1_00000.png
        image_paths.sort(key=lambda x: int(x.split("/")[-1].split('_')[2].split('.')[0]))
    if type == "caption":
        # no sorting needed
        pass
    if type == "ground_truth":
        # names are frame0.png
        image_paths.sort(key=lambda x: int(x.split("/")[-1].split('frame')[1].split('.')[0]))
    
    
    return image_paths, out_of_range


def get_captions_prescan(dir):
    caption = []
    with open(dir, 'r') as file:
        data = file.read()
        caption.append(data)
    return caption

def calculate_the_indexes_of_prescan(len_frames_ground_truth, len_frames_prescan, indexes):
    # Correct the use of max_frames and min_frames
    mapped_indexes = []
    min_frames = min(len_frames_ground_truth, len_frames_prescan)
    max_frames = max(len_frames_ground_truth, len_frames_prescan)

    for frame_idx in indexes:
        # Correct the calculation of current_frame
        current_frame = min(int(frame_idx * (max_frames / min_frames)), max_frames - 1)
        mapped_indexes.append(current_frame)

    return mapped_indexes

    

prescan_root_dir = "/mnt/e/13_Jasper_diffused_samples/complete_dataset/prescan_normal_output/new_scenes_for_new_model"
nuscene_root_dit = "/mnt/e/13_Jasper_diffused_samples/nuscene_frames"
scene_names = os.listdir(prescan_root_dir)

# Create an error.txt file
error_file = open("error.txt", "w")


for scene in scene_names:

    try:
        # First we need to try and get the paths to all the folders and check if there are files in them
        # If there are no files in the folder, we need to skip it

        ground_truth_path = os.path.join(nuscene_root_dit, scene) 
        caption_path = os.path.join(prescan_root_dir, scene, "caption") # should contain one file
        reference_image_path = os.path.join(prescan_root_dir, scene, "rgb")
        conditioning_images_one_path = os.path.join(prescan_root_dir, scene, "seg")
        conditioning_images_two_path = os.path.join(prescan_root_dir, scene, "edges")

        # in the reference_image, conditioning_images_one_path, conditioning_images_two_path, we need to check if there are files in the folder and the same amount
        # get the files in the folders
        ref_files = len(os.listdir(reference_image_path))
        cond_files_one = len(os.listdir(conditioning_images_one_path))
        cond_files_two = len(os.listdir(conditioning_images_two_path))
        cap_files = len(os.listdir(caption_path))
        ground_truth_files = len(os.listdir(ground_truth_path))

        if ref_files == 0 or cond_files_one == 0 or cond_files_two == 0:
            print(f"Skipping {scene} because there are no files in the folders")
            raise ValueError("There are no files in the folders")
        if ref_files != cond_files_one or ref_files != cond_files_two or cond_files_one != cond_files_two:
            print(f"Skipping {scene} because there are not the same amount of files in the folders")
            raise ValueError("The conditioning images do not have the same amount of files")
        if cap_files != 1:
            print(f"Skipping {scene} because there is not precisely one file in the folder")
            raise ValueError("The caption does not have the same amount of files")
        if ground_truth_files == 0:
            print(f"Skipping {scene} because there are no files in the ground_truth folder")
            raise ValueError("There are no files in the ground_truth folder")

        # We need to take bites of 25 frames from the ground_truth folder

        number_of_segments = len(os.listdir(ground_truth_path)) // 14 

        # Now we need to get the corresponding images for each segment
        for i in range(number_of_segments):
            offset = 5
            indexes_nuscenes = range(i * 14 + offset, (i + 1) * 14 + offset)
            ground_truth_indexes = range(i * 14 , (i + 1) * 14 )
            indexes_prescan = calculate_the_indexes_of_prescan(ground_truth_files, cond_files_one, indexes_nuscenes)                 # add the indexes that need to be calculated
            index_reference_image = [indexes_prescan[0]]

            # get the images path
            data_ground_truth, out_of_range1 = get_image_paths_prescan(ground_truth_path, ground_truth_indexes, "ground_truth")
            data_reference_image_path, out_of_range2 = get_image_paths_prescan(reference_image_path, index_reference_image, "rgb")
            data_conditioning_images_one, out_of_range3 = get_image_paths_prescan(conditioning_images_one_path, indexes_prescan, "seg")
            data_conditioning_images_two, out_of_range4 = get_image_paths_prescan(conditioning_images_two_path, indexes_prescan, "edges")
            data_caption = get_captions_prescan(caption_path + "/" + "caption.txt")

            if out_of_range1 or out_of_range2 or out_of_range3 or out_of_range4:
                print(f"Skipping {scene} because the indexes are out of range")
                raise ValueError("The indexes are out of range")

            # double check if conditioning_images_one and conditioning_images_two have the same lenght
            if len(data_conditioning_images_one) != len(data_conditioning_images_two):
                print(f"Skipping {scene} because there are not the same amount of files in the folders")
                raise ValueError("The conditioning images do not have the same amount of files")

            # check that reference_image_path and caption have a length of 1
            if len(data_reference_image_path) != 1 or len(data_caption) != 1:
                print(f"Skipping {scene} because there is not precisely one file in the folder")
                raise ValueError("The reference image or the caption do not have the same amount of files")
            

            # Now all the values need to appended to the dictornary
            data_dict['scene_name'].append(scene)
            data_dict['ground_truth'].append(data_ground_truth)
            data_dict['caption'].append(data_caption)
            data_dict['conditioning_images_one'].append(data_conditioning_images_one)
            data_dict['conditioning_images_two'].append(data_conditioning_images_two)
            data_dict['reference_image'].append(data_reference_image_path)

            # stop the loop
        #     if i == 2:
        #         break 
        # break

    except Exception as e:  
        # Write to the error file
        print(e)
        error_file.write(f"{scene} caused an error: {e}\n")




# Save the data_dict to a json file
with open('complete_data_paths.json', 'w') as file:
    json.dump(data_dict, file, indent=4)



/home/wisley/miniconda3/envs/wsl_diffusers/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Errno 2] No such file or directory: '/mnt/e/13_Jasper_diffused_samples/nuscene_frames/scene-0103'
[Errno 2] No such file or directory: '/mnt/e/13_Jasper_diffused_samples/nuscene_frames/scene-0361'
[Errno 2] No such file or directory: '/mnt/e/13_Jasper_diffused_samples/nuscene_frames/scene-0378'
[Errno 2] No such file or directory: '/mnt/e/13_Jasper_diffused_samples/nuscene_frames/scene-0401'
[Errno 2] No such file or directory: '/mnt/e/13_Jasper_diffused_samples/nuscene_frames/scene-0420'
[Errno 2] No such file or directory: '/mnt/e/13_Jasper_diffused_samples/nuscene_frames/scene-0421'
[Errno 2] No such file or directory: '/mnt/e/13_Jasper_diffused_samples/nuscene_frames/scene-0426'
[Errno 2] No such file or directory: '/mnt/e/13_Jasper_diffused_samples/nuscene_frames/scene-0449'
[Errno 2] No such file or directory: '/mnt/e/13_Jasper_diffused_samples/nuscene_frames/scene-0535'
[Errno 2] No such file or directory: '/mnt/e/13_Jasper_diffused_samples/nuscene_frames/scene-0541'
[Errno 2] 